In [1]:
#https://graph.microsoft.com/v1.0/sites/{site_id}/drives/{drive_id}/list/items?expand=fields

#https://graph.microsoft.com/v1.0/me/drive/root:/Pictures/Familiebilder/Videoer gamle:/children

#https://graph.microsoft.com/v1.0/me/drive/root:/Pictures/Familiebilder/Videoer gamle:/list/items?expand=fields

In [2]:
with open("token.txt", "r") as file:
    token = file.read()

print(token)

Bearer EwCYA8l6BAAUbDba3x2OMJElkF7gJ4z/VbCPEz0AAaAbBmJP8fFYA726UecWNkgTjqNn5OegPc6KlTPrsD3O5a8WAsy4w7Cbh9P65bolVGstwBlfd5IrhnSdTEw8QdjKl2mxsFCO2C73vmK49jbrWRdsWcOZqm+F2wZMnQIvaHpCoKAU1oVBbhgqWvT+ICVRErzgx+88T8b4ci6jIpAUwg8GuL0OoS/r6xPC/gvtZluo2/jmq8olD7SR3KcUJSw9tkUwpABQLY0lGQeVVoodVCUFM3CPtOUPe9tP7eykezKbWPJQTuVhQraKiVTW9XsrPJHOxt460AO/rTFJJ2pXRRjGvpZj01G4Ne7Tk9mLqP6Ac4OF5YLfWOIASPBbPlgQZgAAECGkoMnUbJ27CW4xVUDyldpgAhazK2RQgdJCs2P87quEv2kmgLh+HKVgE066xI1uGJ3FxxSbZOQMXiW5SN4FEay/iFVAeGzoDYN8JHCiTNJvcGzOqpC3zZS/kStNcSdTE7TFt/Fzw5ww5BiMBW/BPE7AEuON+7lqyH4n+ZqUIhEf3IrdJfxm9OFgK46g4Uo5CQQqK6VGGbVwA5gJTQjeAdvwk2nEXa0ntPI9qXR9MUCJHKI4SIyLshBTax5XCY2tNbpix+LipRi6AxmiVUjojz9bCMcL0zcjnsMVV2QCSBW6EDGd+C6t5LEQSoRig0iaWvtUqRi9kPs3W9ZQmx+OGKSiaI1zp5BDsP+QCKnh9sX0IHHk2BXHlFo59Ary15MSTssTDVMzdRTIhPBM8z06SWNuM/Zqd8CCCTKAD1kxFVR18fvh8yx8wPXUPkB2Fm4sWlu5f0HAIgYWI5xnVOgLpiRu9rO//Ov2e1KYMy+vKachMThgP/2P9illK5L4KOR3N+3p4dU6NpkF0o2Z5frUer9W+zUCGRdAy0Yrx8wRBl324goJvojfWFWq6CTRdhXDDRjEoriCZzcTGO4d6Hear4tr74ZY9

In [3]:
#async def get_items(): #-> AsyncGenerator[ListItem, None]:
import aiohttp

item_ids = ["47575C443A523D3A!76020"]
new_item_ids = []
final_item_ids = []
async with aiohttp.ClientSession(
    headers={
        "Content-Type": "application/json",
        "Accept": "application/json",
    }
) as http_client:
    while item_ids:
        for item_id in item_ids:
            async with http_client.get(
                url=f"https://graph.microsoft.com/v1.0/me/drive/items/{item_id}/children",
                headers={"Authorization": f"{token}"},
            ) as response:
                response.raise_for_status()
                msg = await response.json()
                items = msg["value"]

            for item in items:
                #print(item["eTag"])
                #print(item["name"])
                #print(item["id"])
                #print()
                if item["name"].endswith(".VOB"):
                    final_item_ids.append((item["id"], item["name"], item["parentReference"]["id"]))
                elif "file" not in item:
                    new_item_ids.append(item["id"])
        item_ids = new_item_ids
        new_item_ids = []


In [4]:
#len(item_ids)

In [5]:
#len(final_item_ids)

In [6]:
#final_item_ids = final_item_ids[0:3]
len(final_item_ids)

52

In [8]:
import os
import aiohttp
import aiofiles
import asyncio
from tqdm.auto import tqdm
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

output_dir = "vob_files"
CONCURRENT_DOWNLOADS = 3  # Limit concurrent downloads

# Semaphore to limit concurrent downloads
semaphore = asyncio.Semaphore(CONCURRENT_DOWNLOADS)

async def download_file(http_client, item_id, item_name, item_parent_id, token, output_dir):
    async with semaphore:  # Limit concurrent downloads
        try:
            async with http_client.get(
                url=f"https://graph.microsoft.com/v1.0/me/drive/items/{item_id}/content",
                headers={"Authorization": f"{token}"}
            ) as response:
                response.raise_for_status()

                # Get content length for progress bar
                total_size = int(response.headers.get("Content-Length", 0))
                file_path = os.path.join(output_dir, f"{item_parent_id}-{item_name}")
                os.makedirs(output_dir, exist_ok=True)  # Ensure output directory exists

                # Stream the content in chunks
                chunk_size = 32768  # 32 KB
                downloaded_size = 0

                with tqdm(
                    total=total_size,
                    unit="B",
                    unit_scale=True,
                    unit_divisor=1024,
                    desc=f"Downloading {item_name}",
                    position=1,
                ) as progress_bar:
                    async with aiofiles.open(file_path, "wb") as f:
                        async for chunk in response.content.iter_chunked(chunk_size):
                            if chunk:
                                await f.write(chunk)
                                downloaded_size += len(chunk)
                                progress_bar.update(len(chunk))

                logger.info(f"Downloaded: {item_parent_id}-{item_name}")
        except aiohttp.ClientError as e:
            logger.error(f"Error downloading {item_parent_id}-{item_name}: {e}")

async def main(final_item_ids, token):
    async with aiohttp.ClientSession(
        headers={
            "Content-Type": "application/json",
            "Accept": "application/json",
        }
    ) as http_client:
        # Create download tasks
        download_tasks = [
            download_file(http_client, item_id, item_name, item_parent_id, token, output_dir)
            for item_id, item_name, item_parent_id in final_item_ids
        ]

        # Run all tasks concurrently with asyncio.gather
        await tqdm.gather(*download_tasks)

# Example usage
# asyncio.run(main(final_item_ids, token))

await main(final_item_ids, token)

  0%|          | 0/52 [00:00<?, ?it/s]

INFO:__main__:Downloaded: 47575C443A523D3A!76060-VIDEO_TS.VOB
INFO:__main__:Downloaded: 47575C443A523D3A!76093-VIDEO_TS.VOB


In [ ]:
True

In [ ]:
# import os

# input_path = "vob_files"
# output_path = "mp4_files"
# for filename in tqdm(os.listdir(input_path), desc="Converting files from VOB to MP4"):
#     if (filename.endswith(".VOB")): #or .avi, .mpeg, whatever.
#         os.system(f"ffmpeg -y -i {input_path}/{filename} {output_path}/{filename[:-3]}mp4 > /dev/null")
#     else:
#         continue

In [ ]:
import re
import subprocess
from tqdm.auto import tqdm
import os

# Function to run FFmpeg and capture its output
def run_ffmpeg(input_file, output_file):
    # The FFmpeg command to run
    command = [
        'ffmpeg',
        '-y',
        '-i', input_file,
        '-c:v', 'libx264',  # Example codec, change as needed
        '-c:a', 'aac',      # Example codec for audio, change as needed
        output_file
    ]
    
    # Run the FFmpeg process and capture the output
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    
    # Read the FFmpeg output line by line
    for line in process.stderr:
        yield line

# Function to extract time from FFmpeg output using regex
def extract_time(ffmpeg_output):
    time_pattern = r"time=(\d{2}:\d{2}:\d{2}\.\d{2})"
    times = re.findall(time_pattern, ffmpeg_output)
    return times

# Function to get the total duration of the video
def get_video_duration(input_file):
    # Run FFmpeg probe command to get the total duration of the video
    probe_command = ['ffmpeg', '-i', input_file]
    probe_process = subprocess.Popen(probe_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    for line in probe_process.stderr:
        if 'Duration' in line:
            # Extract the duration from the FFmpeg probe output
            duration_match = re.search(r"Duration: (\d{2}):(\d{2}):(\d{2})\.(\d{2})", line)
            if duration_match:
                hours = int(duration_match.group(1))
                minutes = int(duration_match.group(2))
                seconds = int(duration_match.group(3))
                milliseconds = int(duration_match.group(4))
                total_duration = hours * 3600 + minutes * 60 + seconds + milliseconds / 100
                return total_duration
    return None

# Function to track progress using tqdm
def track_progress(input_file, output_file):
    total_duration = get_video_duration(input_file)

    if total_duration is None:
        print("Could not extract video duration.")
        return

    # Initialize tqdm progress bar
    with tqdm(total=total_duration, unit="s", desc="Progress", position=1) as pbar: #ncols=100
        # Run FFmpeg to process the video
        for line in run_ffmpeg(input_file, output_file):
            # Extract time from FFmpeg output
            times = extract_time(line)
            for t in times:
                # Convert time to seconds
                h, m, s = map(float, t.split(':'))
                current_time = h * 3600 + m * 60 + s
                pbar.n = current_time
                pbar.last_print_n = current_time
                pbar.update(0)  # Update the progress bar without moving it forward automatically
                pbar.set_postfix(time=t)


input_path = "vob_files"
output_path = "mp4_files"

for filename in tqdm(os.listdir(input_path), desc="Converting files from VOB to MP4", position=0):
    if (filename.endswith(".VOB")): #or .avi, .mpeg, whatever.
        input_file = f"{input_path}/{filename}"
        output_file = f"{output_path}/{filename[:-4]}.mp4"
        track_progress(input_file, output_file)

Converting files from VOB to MP4:   0%|          | 0/3 [00:00<?, ?it/s]

Could not extract video duration.


Progress:   0%|          | 0/1791.3 [00:00<?, ?s/s]

Progress:   0%|          | 0/465.0 [00:00<?, ?s/s]

In [ ]:
import os
import aiohttp
import aiofiles
import asyncio
from tqdm.auto import tqdm
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

input_path = "vob_files"
output_path = "mp4_files"

# Semaphore to limit concurrent uploads
CONCURRENT_UPLOADS = 5  # Adjust as needed
semaphore = asyncio.Semaphore(CONCURRENT_UPLOADS)

async def upload_chunk(http_client, upload_url, headers, chunk_data, retries=3):
    for attempt in range(retries):
        try:
            async with http_client.put(
                url=upload_url,
                headers=headers,
                data=chunk_data
            ) as response:
                response.raise_for_status()
                return
        except aiohttp.ClientError as e:
            if attempt < retries - 1:
                await asyncio.sleep(2 ** attempt)  # Exponential backoff
                continue
            logger.error(f"Failed to upload chunk after {retries} attempts: {e}")
            raise

async def upload_file(fname, token, output_path):
    async with semaphore:  # Limit concurrent uploads
        try:
            item_parent_id, filename = fname.split("-")
        except ValueError:
            logger.error(f"Invalid file name format: {fname}. Skipping.")
            return

        async with aiohttp.ClientSession() as http_client:
            request_body = {
                "item": {
                    "description": "a large file",
                    "name": filename
                }
            }
            async with http_client.post(
                url=f"https://graph.microsoft.com/v1.0/me/drive/items/{item_parent_id}:/{filename}:/createUploadSession",
                headers={"Authorization": f"{token}"},
                json=request_body
            ) as response_upload_session:
                response_upload_session.raise_for_status()
                upload_url = (await response_upload_session.json())["uploadUrl"]

            try:
                async with aiofiles.open(f'{output_path}/{fname}', 'rb') as upload:
                    total_file_size = os.path.getsize(f'{output_path}/{fname}')
                    chunk_size = 327680
                    chunk_number = (total_file_size + chunk_size - 1) // chunk_size

                    for counter in tqdm(range(chunk_number), desc=f"Uploading {filename}", position=1):
                        chunk_data = await upload.read(chunk_size)
                        start_index = counter * chunk_size
                        end_index = start_index + len(chunk_data)

                        headers = {
                            "Content-Length": f"{len(chunk_data)}",
                            "Content-Range": f"bytes {start_index}-{end_index - 1}/{total_file_size}"
                        }

                        await upload_chunk(http_client, upload_url, headers, chunk_data)
                    logger.info(f"File {filename} upload complete.")
            except FileNotFoundError:
                logger.error(f"File {fname} not found in {output_path}. Skipping.")
            except PermissionError:
                logger.error(f"Permission denied for file {fname}. Skipping.")

async def main():
    #token = "your_token_here"
    #if not token:
    #    raise ValueError("Authorization token is missing or invalid.")

    # Collect all filenames
    file_list = os.listdir(output_path)

    # Create upload tasks
    upload_tasks = [
        upload_file(fname, token, output_path)
        for fname in tqdm(file_list, desc="Preparing uploads", position=0)
    ]

    # Run tasks concurrently
    await tqdm.gather(*upload_tasks)


await main()

Preparing uploads:   0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Uploading VTS_01_2.mp4:   0%|          | 0/1729 [00:00<?, ?it/s]

Uploading VTS_01_1.mp4:   0%|          | 0/1618 [00:00<?, ?it/s]

INFO:__main__:File VTS_01_1.mp4 upload complete.
INFO:__main__:File VTS_01_2.mp4 upload complete.
